In [3]:
import streamlit as st

with st.echo():
    from selenium import webdriver
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.chrome.service import Service
    from webdriver_manager.chrome import ChromeDriverManager

    @st.experimental_singleton
    def get_driver():
        return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

    options = Options()
    options.add_argument('--disable-gpu')
    options.add_argument('--headless')

    driver = get_driver()
    driver.get("http://example.com")

    st.code(driver.page_source)

2024-02-01 22:04:18.154 
  command:

    streamlit run /Users/han.med/Coding/zillow-dealfinder/v_env/lib/python3.11/site-packages/ipykernel_launcher.py [ARGUMENTS]


ModuleNotFoundError: No module named 'webdriver_manager'

In [4]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

#request_headers = {
#            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
#            'accept-encoding': 'gzip, deflate, br',
#            'accept-language': 'en-US,en;q=0.8',
#            'upgrade-insecure-requests': '1',
#            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
#        }

url = input('Enter - ')
html = urllib.request.urlopen('url', context=ctx).read()
soup = BeautifulSoup(html, 'html.parser')

# Retrieve all of the anchor tags
tags = soup('a')
for tag in tags:
    print(tag)

ValueError: unknown url type: 'url'